In [1249]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import matplotlib.colors as colors
from scipy import constants as const
%matplotlib qt
spectra=nanonis.biasSpectroscopy()
Eh=const.physical_constants['Hartree energy in eV'][0]

# File Management

In [2]:
#data=pd.read_excel("C:/Users/jonor/Documents/GitHub/nanonis/Order.xls")
data=pd.read_excel("C:/Users/jortuzar/Documents/Git/nanonis/Jon/Order.xls")


In [93]:
Bfield_files=data.iloc[:,2]
Bfields=data.iloc[:,3]
size_dat=data.iloc[:,7]
height_dat=data.iloc[:,8]
temp=data.iloc[:,6]
temp_files=data.iloc[:,5]
josephson=data.iloc[:,4]
path=data.iloc[:,11]
Zapproach_fil=data.iloc[:,1]

In [674]:
#dat=[ 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24]
dat=[ 0,1,2,3,4,5,6,7, 8, 11, 12, 13, 14, 15, 16, 17, 19, 21, 23, 24,32]
T=[]
Tfiles=[]
Bfiles=[]
Jfiles=[]
size=[]
height=[]
Bfiles=[]
B=[]
for j in dat:
    #T.append([float(i) for i in temp[j].split(',')])
    #Tfiles.append([path[j]+i for i in temp_files[j].split(',')])
    size.append(size_dat[j])
    height.append(height_dat[j])
    B.append([int(i) for i in Bfields[j].split(',')])
    Bfiles.append([path[j]+i for i in Bfield_files[j].split(',')])
    Jfiles.append([path[j]+i for i in josephson[j].split(',')])


for j in range(len(B)):
    list1, list2 = zip(*sorted(zip(B[j], Bfiles[j])))
    B[j]=np.array(list1)
    Bfiles[j]=np.array(list2)
size=np.array(size)
height=np.array(height)

Zapproach=[]
Zapproach.append([path[32]+i for i in Zapproach_fil[32].split(',')])

# Some definitions

In [1173]:
def find_nearest(array,value):
        idx = (np.abs(array-value)).argmin()
        return idx

def fast_spec(file):
        spectra.load(file)
        x=np.array(spectra.bias)
        spectra.normalizeRange_symm([x[5],x[0]])
        y=np.array(spectra.conductance)
        return x,y

def fast_spec_drift(file):
        spectra.load(file)
        x=np.array(spectra.bias)
        spectra.normalizeRange_symm([x[5],x[0]])
        y=np.array(spectra.conductance)
        a=np.mean(y[[0,1,2,3,4,5]])
        b=np.mean(y[[-1,-2,-3,-4,-5,-6]])
        f= lambda t: (a-b)*t/(-x[2]+x[-3])+a+x[2]*(a-b)/(-x[2]+x[-3])
        y=y*f(x)
        return x,y

def fast_cur(file):
        spectra.load(file)
        x=np.array(spectra.bias)
        y=np.array(spectra.current)
        return x,y

def fast_I(file):
        spectra.load(file)
        x=np.array(spectra.bias)
        y=np.array(spectra.biasVI_b)
        z=np.array(spectra.biasVI_f)
        w=np.array(spectra.current)
        a=np.abs((w[0]-w[-1])/(x[0]-x[-1]))*1e6
        x=x*a
        return x,y,z,w,a


color_list=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

## No Pof E

In [1148]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)
    return (x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)
    return (x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    I=-np.gradient(I)
    return I/np.abs(I[0])


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return t/t[0]

## P of E

In [919]:
def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def f(t,x,R,C,T):
    x=np.linspace(0.01,5,2000)
    dx=x[1]-x[0]
    ReZ=R/(1+(x*R*C)**2)
    a,b=np.meshgrid(x,t)
    c=a*b
    return  np.dot(np.cos(c)-1,2*(ReZ/x)/np.tanh(x/(2*T)))*dx- np.dot(1j*np.sin(c) ,2*(ReZ/x) )*dx


def P(R,C1,C2,T,n):
    dis=1
    N=3000
    C=C1*C2/(C1+C2)
    if n==1:
        k=C2/(C1+C2)
    else:
        k=C1/(C1+C2)
    t=np.linspace(-dis,dis,N)
    x=np.linspace(0.01,300,1000)
    freq=np.fft.fftfreq(N,d=2*dis/N)
    a=np.argsort(freq)
    y=np.real(np.fft.fft(np.exp(k*f(t,x,R,C,T))))
    t=y[a]
    return np.flip(np.sort(freq)),np.abs(t)

def Gamma1(V,R1,R2,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)
    w,PoE=P(R1+R2,C1,C2,T,1)
    w=w/(2*np.pi*27)
    dw=w[1]-w[0]
    a,b=np.meshgrid(w,x)
    c=-a+b
    return -np.dot(c/(1-np.exp(-c/T)),PoE)*dw/R1

def Gamma2(V,R1,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)
    w,PoE=P(R1+R2,C1,C2,T,2)
    w=w/(2*np.pi*27)
    dw=w[1]-w[0]
    a,b=np.meshgrid(w,x)
    c=-a+b
    return  -np.dot(c/(1-np.exp(-c/T)),PoE)*dw/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,R2,C1,C2,-n,Q0,T)+Gamma2(-V,R1,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,R2,C1,C2,n+1,-Q0,T)+Gamma2(V,R1,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,R2,C1,C2,-n,-Q0,T)+Gamma2(V,R1,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,R2,C1,C2,n+1,Q0,T)+Gamma2(-V,R1,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,R2,C1,C2,Q0,T,n):
    return Gamma1(V,R1,R2,C1,C2,n,Q0,T)-Gamma1(-V,R1,R2,C1,C2,-n,-Q0,T)

def G2n(V,R1,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R1,R2,C1,C2,n,Q0,T)-Gamma2(-V,R1,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,R2,C1,C2,Q0,T,-i)
    I=np.gradient(I)
    return I/np.abs(I[0])


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R1,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R1,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R1,R2,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return t/t[0]

## Improoved program

In [917]:
En=np.zeros(25)

def f(t,x,R,C,T):
    x=np.linspace(0.01,5,2000)
    dx=x[1]-x[0]
    ReZ=R/(1+(x*R*C)**2)
    a,b=np.meshgrid(x,t)
    c=a*b
    return  np.dot(np.cos(c)-1,2*(ReZ/x)/np.tanh(x/(2*T)))*dx- np.dot(1j*np.sin(c) ,2*(ReZ/x) )*dx


def P(R,C1,C2,T,n):
    dis=1
    N=3000
    C=C1*C2/(C1+C2)
    if n==1:
        k=C2/(C1+C2)
    else:
        k=C1/(C1+C2)
    t=np.linspace(-dis,dis,N)
    x=np.linspace(0.01,300,1000)
    freq=np.fft.fftfreq(N,d=2*dis/N)
    a=np.argsort(freq)
    y=np.real(np.fft.fft(np.exp(k*f(t,x,R,C,T))))
    t=y[a]
    return np.flip(np.sort(freq)),np.abs(t)


def init_(R1,R2,C1,C2,Q0,T):
    f1,a1=P(R1+R2,C1,C2,T,1)
    global freq1
    freq1=f1/(2*np.pi*27)
    global PoE1
    PoE1=a1
    f2,a2=P(R1+R2,C1,C2,T,2)
    global freq2
    freq2=f2/(2*np.pi*27)
    global PoE2
    PoE2=a2

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,R2,C1,C2,n,Q0,T):
    x=E1(C1,C2,V,n,Q0)
    w=freq1
    dw=w[1]-w[0]
    a,b=np.meshgrid(w,x)
    c=-a+b
    return -np.dot(c/(1-np.exp(-c/T)),PoE1)*dw/R1

def Gamma2(V,R1,R2,C1,C2,n,Q0,T):
    x=E2(C1,C2,V,-n,-Q0)
    w=freq2
    dw=w[1]-w[0]
    a,b=np.meshgrid(w,x)
    c=-a+b
    return  -np.dot(c/(1-np.exp(-c/T)),PoE2)*dw/R2



def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,R2,C1,C2,-n,Q0,T)+Gamma2(-V,R1,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,R2,C1,C2,n+1,-Q0,T)+Gamma2(V,R1,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,R2,C1,C2,-n,-Q0,T)+Gamma2(V,R1,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,R2,C1,C2,n+1,Q0,T)+Gamma2(-V,R1,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,R2,C1,C2,Q0,T,n):
    return Gamma1(V,R1,R2,C1,C2,n,Q0,T)-Gamma1(-V,R1,R2,C1,C2,-n,-Q0,T)

def G2n(V,R1,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R1,R2,C1,C2,n,Q0,T)-Gamma2(-V,R1,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    init_(R1,R2,C1,C2,Q0,T)
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,R2,C1,C2,Q0,T,-i)
    I=np.gradient(I)
    return I/np.abs(I[0])


def current2(V,R1,R2,C1,C2,Q0,T):
    init_(R1,R2,C1,C2,Q0,T)
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R1,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R1,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R1,R2,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return t/t[0]

## Improoved Improoved Program

In [1491]:
En=[0 for i in range(25)]
Mn=[0 for i in range(25)]
def f(t,x,R,C,T):
    x=np.linspace(0.01,5,2000)
    dx=x[1]-x[0]
    ReZ=R/(1+(x*R*C)**2)
    a,b=np.meshgrid(x,t)
    c=a*b
    return  np.dot(np.cos(c)-1,2*(ReZ/x)/np.tanh(x/(2*T)))*dx- np.dot(1j*np.sin(c) ,2*(ReZ/x) )*dx


def P(R,C1,C2,T,n):
    dis=1
    N=2000
    C=C1*C2/(C1+C2)
    if n==1:
        k=C2/(C1+C2)
    else:
        k=C1/(C1+C2)
    t=np.linspace(-dis,dis,N)
    x=np.linspace(0.01,10,100)
    freq=np.fft.fftfreq(N,d=2*dis/N)
    a=np.argsort(freq)
    y=np.real(np.fft.fft(np.exp(k*f(t,x,R,C,T))))
    t=np.abs(y[a])
    return np.flip(np.sort(freq)),t/sum(t)


def init_(R1,R2,C1,C2,Q0,T):
    f1,a1=P(R1+R2,C1,C2,T*Eh,1)
    global freq1
    freq1=f1/(2*np.pi*27)
    global PoE1
    PoE1=a1
    f2,a2=P(R1+R2,C1,C2,T*Eh,2)
    global freq2
    freq2=f2/(2*np.pi*27)
    global PoE2
    PoE2=a2

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,R2,C1,C2,n,Q0,T):
    x=E1(C1,C2,V,n,Q0)
    w=freq1
    dw=w[1]-w[0]
    a,b=np.meshgrid(w,x)
    c=-a+b
    return -np.dot(c/(1-np.exp(-c/T)),PoE1)*dw/R1

def Gamma2(V,R1,R2,C1,C2,n,Q0,T):
    x=E2(C1,C2,V,-n,-Q0)
    w=freq2
    dw=w[1]-w[0]
    a,b=np.meshgrid(w,x)
    c=-a+b
    return  -np.dot(c/(1-np.exp(-c/T)),PoE2)*dw/R2



def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,R2,C1,C2,-n,Q0,T)+Gamma2(-V,R1,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,R2,C1,C2,n+1,-Q0,T)+Gamma2(V,R1,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,R2,C1,C2,-n,-Q0,T)+Gamma2(V,R1,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,R2,C1,C2,n+1,Q0,T)+Gamma2(-V,R1,R2,C1,C2,-n-1,-Q0,T))
    global En
    En[n]= en
    global Mn
    Mn[n]= mn
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    n=[0,1]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        a.append(En[i])
        b.append(Mn[i])
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,R2,C1,C2,Q0,T,n):
    return Gamma1(V,R1,R2,C1,C2,n,Q0,T)-Gamma1(-V,R1,R2,C1,C2,-n,-Q0,T)

def G2n(V,R1,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R1,R2,C1,C2,n,Q0,T)-Gamma2(-V,R1,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    init_(R1,R2,C1,C2,Q0,T)
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,R2,C1,C2,Q0,T,-i)
    I=np.gradient(I)
    return I/np.abs(I[0])


def current2(V,R1,R2,C1,C2,Q0,T):
    init_(R1,R2,C1,C2,Q0,T)
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R1,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R1,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R1,R2,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return t/t[0]

## No PoF Superconductivity

In [1333]:
def init_(V,delta_t,Delta,T,eta):
    global x_int
    x_int=np.linspace(-4*np.max(V),4*np.max(V),5000)

    global bcsf
    bcsf=bcs(delta_t,x_int,eta)*fermi(T,x_int)

    global bcsif_even
    bcsif_even=bcs_i(Delta,x_int,eta,0)*fermi(T,x_int)

    global bcsif_odd
    bcsif_odd=bcs_i(Delta,x_int,eta,1)*fermi(T,x_int)

def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1
        

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0,Delta):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0,Delta):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    a,b=np.meshgrid(x_int,E1(C1,C2,V,n,Q0,Delta))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n)*(fermi(T,-t)),bcsf )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    a,b=np.meshgrid(x_int,E2(C1,C2,V,-n,-Q0,Delta))
    t=a+b
    if np.mod(n,2)==0:
        return np.dot(bcs(delta_s,t,eta)*(fermi(T,-t)),bcsif_even)/R2
    else:
        return np.dot(bcs(delta_s,t,eta)*(fermi(T,-t)),bcsif_odd)/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.01 and p_n<0.01:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(i+1):
            temp1*=a[j]     
            temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    init_(V,delta_t,Delta,T,eta)
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    init_(V,delta_t,Delta,T,eta)
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


## Improoved No PoF Superconductivity

In [1568]:
def initD_(V,delta_t,Delta,T,eta):
    global EnD
    EnD=[0 for i in range(25)]
    global MnD
    MnD=[0 for i in range(25)]
    global x_int
    x_int=np.linspace(-4*np.max(V),4*np.max(V),5000)

    global bcsf
    bcsf=bcs(delta_t,x_int,eta)*fermi(T,x_int)

    global bcsif_even
    bcsif_even=bcs_i(Delta,x_int,eta,0)*fermi(T,x_int)

    global bcsif_odd
    bcsif_odd=bcs_i(Delta,x_int,eta,1)*fermi(T,x_int)

def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1
        

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    a,b=np.meshgrid(x_int,E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n)*(fermi(T,-t)),bcsf )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    a,b=np.meshgrid(x_int,E2(C1,C2,V,-n,-Q0))
    t=a+b
    if np.mod(n,2)==0:
        return np.dot(bcs(delta_s,t,eta)*(fermi(T,-t)),bcsif_even)/R2
    else:
        return np.dot(bcs(delta_s,t,eta)*(fermi(T,-t)),bcsif_odd)/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    global En
    EnD[n]= en
    global Mn
    MnD[n]= mn
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.01 and p_n<0.01:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n+1):
        a.append(EnD[i])
        b.append(MnD[i])
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(i+1):
            temp1*=a[j]     
            temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    initD_(V,delta_t,Delta,T,eta)
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    initD_(V,delta_t,Delta,T,eta)
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


## PoE Superconductivity

In [1567]:
def f(t,x,R,C,T):
    x=np.linspace(0.01,5,2000)
    dx=x[1]-x[0]
    ReZ=R/(1+(x*R*C)**2)
    a,b=np.meshgrid(x,t)
    c=a*b
    return  np.dot(np.cos(c)-1,2*(ReZ/x)/np.tanh(x/(2*T)))*dx- np.dot(1j*np.sin(c) ,2*(ReZ/x) )*dx


def PD(R,C1,C2,T,n,N,Vmax):
    dis=(N/2-1)/(16*np.pi*Vmax*Eh)

    C=C1*C2/(C1+C2)
    if n==1:
        k=C2/(C1+C2)
    else:
        k=C1/(C1+C2)
    t=np.linspace(-dis,dis,N)
    x=np.linspace(0.01,10,100)
    freq=np.fft.fftfreq(N,d=2*dis/N)
    a=np.argsort(freq)
    y=np.real(np.fft.fft(np.exp(k*f(t,x,R,C,T))))
    t=np.abs(y[a])
    return np.flip(np.sort(freq)),t/sum(t)

def initD_(R1,R2,C1,C2,V,delta_t,Delta,T,eta,N):
    global EnD
    EnD=[0 for i in range(25)]
    global MnD
    MnD=[0 for i in range(25)]
    global x_int
    x_int=np.linspace(-4*np.max(V),4*np.max(V),N)
    a,b=np.meshgrid(x_int,x_int)
    t=a+b
    global bcsf
    bcsf=bcs(delta_t,t,eta)*fermi(T,t)

    global bcsif_even
    bcsif_even=bcs_i(Delta,t,eta,0)*fermi(T,t)

    global bcsif_odd
    bcsif_odd=bcs_i(Delta,t,eta,1)*fermi(T,t)

    f1,a1=PD(R1+R2,C1,C2,T*Eh,1,N,np.max(V))
    global freq1
    freq1=f1/(2*np.pi*27)
    global PoE1
    PoE1=a1
    f2,a2=PD(R1+R2,C1,C2,T*Eh,2,N,np.max(V))
    global freq2
    freq2=f2/(2*np.pi*27)
    global PoE2
    PoE2=a2

def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1
        

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    a,b=np.meshgrid(x_int,E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot(bcs_i(Delta,t,eta,n)*(fermi(T,-t)),np.dot(bcsf,PoE1))/R1


def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    a,b=np.meshgrid(x_int,E2(C1,C2,V,-n,-Q0))
    t=a+b
    if np.mod(n,2)==0:
        return np.dot(bcs_i(Delta,t,eta,n)*(fermi(T,-t)),np.dot(bcsif_even,PoE1))/R2
    else:
        return np.dot(bcs_i(Delta,t,eta,n)*(fermi(T,-t)),np.dot(bcsif_odd,PoE1))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    global En
    EnD[n]= en
    global Mn
    MnD[n]= mn
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.01 and p_n<0.01:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,n):
    a=[]
    b=[]
    for i in range(n+1):
        a.append(EnD[i])
        b.append(MnD[i])
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(i+1):
            temp1*=a[j]     
            temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    initD_(R1,R2,C1,C2,V,delta_t,Delta,T,eta,5000)
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    initD_(R1,R2,C1,C2,V,delta_t,Delta,T,eta,5000)
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)



## Analysis

In [1150]:
index=range(8,25)
T_files=[]
T=[]
sizeT=[]
for i in index:
    T_files.append([path[i]+j for j in temp_files[i].split(',')])
    T.append([float(j) for j in temp[i].split(',')])
    sizeT.append(float(size_dat[i]))

In [ ]:
Ec=[]
Q0=[]
ther=[]
h=0
temperature=8.0
size2=[]
a=range(len(T_files))
for i in a:
    for j in range(len(T_files[i])):
        if np.abs(T[i][j]-temperature)<=0.00001:
            x,y=fast_spec_drift(T_files[i][j])
            y=y/y[0]
            ff= lambda x,C2 : current2(x,300,1,0.01,C2,0,0.8)
            values=sc.optimize.curve_fit(ff,1000*x,y,p0=[0.2],bounds=([0.01],[1.5]))[0]
            #yy=coulomb_T(x,values[0],values[1],values[2])
            yy=ff(1000*x,values[0])
            Ec.append(1/values[0])
            #Q0.append(values[1])
            #ther.append(values[2])
            plt.plot(x,yy+h)
            plt.plot(x,y+h)
            h+=1
            size2.append(sizeT[i])
        else:
            continue
        

In [1197]:
EcT=np.array(Ec)
plt.scatter(size2,Ec/2)

In [1188]:
fig,ax=plt.subplots()
fig.subplots_adjust(bottom=0.5)

x=np.linspace(-20,20,100)
i=1
ax.plot(x,current(x,230,1,0.01,0.1,0,0.2))
xx,yy=fast_spec_drift(T_files[i][0])
ax.plot(1000*xx,yy/yy[0])
ax1 = fig.add_axes([0.10, 0.10, 0.65, 0.03])
ax2 = fig.add_axes([0.10, 0.15, 0.65, 0.03])
ax3 = fig.add_axes([0.10, 0.20, 0.65, 0.03])
ax4 = fig.add_axes([0.10, 0.25, 0.65, 0.03])
ax5 = fig.add_axes([0.10, 0.30, 0.65, 0.03])
ax6 = fig.add_axes([0.10, 0.35, 0.65, 0.03])


ax1_s = plt.Slider(ax1,'R1',1,1000,valinit=230)
ax2_s = plt.Slider(ax2,'R2',1,1000,valinit=1)
ax3_s = plt.Slider(ax3,'C1',0.001,0.5,valinit=0.01)
ax4_s = plt.Slider(ax4,'C2',0.001,1,valinit=0.1)
ax5_s = plt.Slider(ax5,'T',0,1,valinit=0.02)
ax6_s = plt.Slider(ax6,'Q0',-1,1,valinit=0.0)



def update(val):
    ax.clear()

    ax.plot(x,current(x,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax6_s.val,ax5_s.val))
    xx,yy=fast_spec_drift(T_files[i][0])
    ax.plot(1000*xx,yy/yy[0])


ax1_s.on_changed(update)
ax2_s.on_changed(update)
ax3_s.on_changed(update)
ax4_s.on_changed(update)
ax5_s.on_changed(update)
ax6_s.on_changed(update)




0

In [760]:
f=open('data.dat','r')
y=[]
x=[]
for i in f:
    y.append(float(i.split(',')[0]))
    x.append(float(i.split(',')[1]))
plt.scatter(x,y)

# B Field

In [1247]:
index=[8,10,11,12,13,14,15,16,17,18,19,21,23,24]
B_files=[]
B=[]
sizeT=[]
T_files=[]
T=[]
sizeT=[]
for i in index:
    B_files.append([path[i]+j for j in Bfield_files[i].split(',')])
    B.append([float(j) for j in Bfields[i].split(',')])
    sizeT.append(float(size_dat[i]))
    T_files.append([path[i]+j for j in temp_files[i].split(',')])
    T.append([float(j) for j in temp[i].split(',')])


In [1566]:
fig,ax=plt.subplots()
fig.subplots_adjust(bottom=0.5)

x=np.linspace(-10,10,200)
i=0
y=currentD(x,300,1,0.01,0.2,0.0,1.3,1.3,0.1,0.1,0.05)
ax.plot(x,y/y[0])
xx,yy=fast_spec_drift(B_files[i][0])
ax.plot(1000*xx,yy/yy[0])
ax1 = fig.add_axes([0.10, 0.10, 0.65, 0.03])
ax2 = fig.add_axes([0.10, 0.15, 0.65, 0.03])
ax3 = fig.add_axes([0.10, 0.20, 0.65, 0.03])
ax4 = fig.add_axes([0.10, 0.25, 0.65, 0.03])
ax5 = fig.add_axes([0.10, 0.30, 0.65, 0.03])
ax6 = fig.add_axes([0.10, 0.35, 0.65, 0.03])
ax7 = fig.add_axes([0.10, 0.40, 0.65, 0.03])


ax1_s = plt.Slider(ax1,'R1',1,1000,valinit=300)
ax2_s = plt.Slider(ax2,'R2',1,1000,valinit=1)
ax3_s = plt.Slider(ax3,'C1',0.001,0.5,valinit=0.01)
ax4_s = plt.Slider(ax4,'C2',0.001,5,valinit=1/Ec[i])
ax5_s = plt.Slider(ax5,'T',0,1,valinit=0.1)
ax6_s = plt.Slider(ax6,'Q0',-1,1,valinit=Q0[i])
ax7_s = plt.Slider(ax7,'Delta',0,1.3,valinit=1.3)


def update(val):
    ax.clear()
    y=currentD(x,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax6_s.val,1.3,ax7_s.val,0.3,ax5_s.val,0.05)
    ax.plot(x,y/y[0])
    xx,yy=fast_spec_drift(B_files[i][0])
    ax.plot(1000*xx,yy/yy[0])


ax1_s.on_changed(update)
ax2_s.on_changed(update)
ax3_s.on_changed(update)
ax4_s.on_changed(update)
ax5_s.on_changed(update)
ax6_s.on_changed(update)
ax7_s.on_changed(update)

0

In [ ]:
Ec=[]
Q0=[]
ther=[]
h=0
size2=[]
a=range(len(B_files))
for i in a:
    x,y=fast_spec_drift(B_files[i][-1])
    y=y/y[0]
    ff= lambda x,C2,Q0 : current2(x,300,1,0.01,C2,Q0,0.1)
    values=sc.optimize.curve_fit(ff,1000*x,y,p0=[0.2,0.0],bounds=([0.01,-0.5],[2.5,0.5]))[0]
    #yy=coulomb_T(x,values[0],values[1],values[2])
    yy=ff(1000*x,values[0],values[1])
    Ec.append(1/values[0])
    Q0.append(values[1])
    #ther.append(values[2])
    print('a')
    plt.plot(x,yy+h)
    plt.plot(x,y+h)
    h+=1
    size2.append(sizeT[i])


In [1561]:
fig,ax=plt.subplots(1,3)
fig.subplots_adjust(bottom=0.35)

x=np.linspace(-10,10,100)
i=1
y=current(x,230,1,0.01,0.1,0,0.1)
ax[0].plot(x,y/y[0])
y=current(x,230,1,0.01,0.1,0,0.8)
ax[1].plot(x,y/y[0])
y=currentD(x,300,1,0.01,0.1,0.0,1.3,1.3,0.1,0.1,0.05)
ax[2].plot(x,y/y[0])

xx,yy=fast_spec_drift(B_files[i][-1])
temp=find_nearest(xx,10*1e-3)
ax[0].plot(1000*xx,yy/yy[temp])
xx,yy=fast_spec_drift(T_files[i][0])
temp=find_nearest(xx,10*1e-3)
ax[1].plot(1000*xx,yy/yy[temp])
xx,yy=fast_spec_drift(B_files[i][0])
temp=find_nearest(xx,10*1e-3)
ax[2].plot(1000*xx,yy/yy[temp])

ax1 = fig.add_axes([0.10, 0.03, 0.65, 0.01])
ax2 = fig.add_axes([0.10, 0.06, 0.65, 0.01])
ax3 = fig.add_axes([0.10, 0.09, 0.65, 0.01])
ax4 = fig.add_axes([0.10, 0.12, 0.65, 0.01])
ax5 = fig.add_axes([0.10, 0.15, 0.65, 0.01])
ax6 = fig.add_axes([0.10, 0.18, 0.65, 0.01])
ax7 = fig.add_axes([0.10, 0.21, 0.65, 0.01])
ax8 = fig.add_axes([0.10, 0.24, 0.65, 0.01])
ax9 = fig.add_axes([0.10, 0.27, 0.65, 0.01])
ax10 = fig.add_axes([0.10, 0.30, 0.65, 0.01])


ax1_s = plt.Slider(ax1,'R1',1,1000,valinit=230)
ax2_s = plt.Slider(ax2,'R2',1,1000,valinit=1)
ax3_s = plt.Slider(ax3,'C1',0.001,0.5,valinit=0.01)
ax4_s = plt.Slider(ax4,'C2',0.001,2,valinit=0.1)
ax5_s = plt.Slider(ax5,'T1',0,1,valinit=0.1)
ax6_s = plt.Slider(ax6,'T2',0,1,valinit=0.8)
ax7_s = plt.Slider(ax7,'Q0',-1,1,valinit=0.0)
ax8_s = plt.Slider(ax8,'A',0.1,1.5,valinit=1.0)
ax9_s = plt.Slider(ax9,'Delta',0.1,1.3,valinit=1.3)
ax10_s = plt.Slider(ax10,'C2_B',0.001,2,valinit=0.1)


def update(val):
    ax[0].clear()
    ax[1].clear()
    ax[2].clear()
    y=current(x,ax1_s.val,ax2_s.val,ax3_s.val,ax10_s.val,ax7_s.val,ax5_s.val)
    ax[0].plot(x,ax8_s.val*y/y[0])
    y=current(x,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax7_s.val,ax6_s.val)
    ax[1].plot(x,y/y[0])
    y=currentD(x,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax7_s.val,1.3,ax9_s.val,0.3,ax5_s.val,0.05)
    ax[2].plot(x,y/y[0])
    xx,yy=fast_spec_drift(B_files[i][-1])
    temp=find_nearest(xx,10*1e-3)
    ax[0].plot(1000*xx,yy/yy[temp])
    xx,yy=fast_spec_drift(T_files[i][0])
    temp=find_nearest(xx,10*1e-3)
    ax[1].plot(1000*xx,yy/yy[temp])
    xx,yy=fast_spec_drift(B_files[i][0])
    temp=find_nearest(xx,10*1e-3)
    ax[2].plot(1000*xx,yy/yy[temp])


ax1_s.on_changed(update)
ax2_s.on_changed(update)
ax3_s.on_changed(update)
ax4_s.on_changed(update)
ax5_s.on_changed(update)
ax6_s.on_changed(update)
ax7_s.on_changed(update)
ax8_s.on_changed(update)
ax9_s.on_changed(update)
ax10_s.on_changed(update)

0

In [1380]:
def f(t,x,R,C,T):
    x=np.linspace(0.01,5,2000)
    dx=x[1]-x[0]
    ReZ=R/(1+(x*R*C)**2)
    a,b=np.meshgrid(x,t)
    c=a*b
    return  np.dot(np.cos(c)-1,2*(ReZ/x)/np.tanh(x/(2*T)))*dx- np.dot(1j*np.sin(c) ,2*(ReZ/x) )*dx


def P(R,C1,C2,T,n):
    N=5000
    dis=(N/2-1)/(4*np.pi*40*Eh)

    C=C1*C2/(C1+C2)
    if n==1:
        k=C2/(C1+C2)
    else:
        k=C1/(C1+C2)
    t=np.linspace(-dis,dis,N)
    x=np.linspace(0.01,10,100)
    freq=np.fft.fftfreq(N,d=2*dis/N)
    a=np.argsort(freq)
    y=np.real(np.fft.fft(np.exp(k*f(t,x,R,C,T))))
    t=np.abs(y[a])
    return np.flip(np.sort(freq)),t/sum(t)

x=np.linspace(-40,40,5000)
fre,p=P(300,0.01,0.3,2.7,1)
fre=fre/(2*np.pi*Eh)
plt.plot(fre,p)
fre[0]

40.00000000000001